# IMPORTS AND REQUIREMENTS

This code was taken and adapted to our problem from the following github repository : 
https://github.com/matterport/Mask_RCNN

In [1]:
##### SKIP IF YOU ARE NOT USING COLLAB #####
############################################



from google.colab import drive
drive.mount('/content/gdrive')
%cd 'gdrive/My Drive/Colab Notebooks' 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [2]:
%cd 'Mask_RCNN-master'

/content/gdrive/My Drive/Colab Notebooks/Mask_RCNN-master


In [0]:
!python3 setup.py install

running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/parallel_model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/__init__.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.linux-x86_64/egg/mrcnn
byte-compiling bu

In [0]:
!pip install Cython
!pip install pycocotools

In [0]:
!pip install imgaug

# SETTINGS FOR THE MASKED RCNN EXECUTION

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [0]:
print(r['rois'][r['class_ids']==15])

[[ 51  64 270 451]]


# ALGORITHM CREATING THE NEW DATASET OF CROPPED IMAGES

In [0]:
#from PIL import Image
k=0
for x in os.listdir('../bird_dataset'):

  if x!='.DS_Store':

    for label in os.listdir('../bird_dataset/'+x):
      
      if label!='.DS_Store':
        
        for im_name in (os.listdir('../bird_dataset/'+x+'/'+label)):
          
          filename =  '../bird_dataset/'+x+'/'+label+'/'+im_name              
          image = skimage.io.imread(filename)
          
          ## don't process grey level images
          if len(image.shape)<3:
            x1 =0
            x2 =image.shape[0]-1
            y1 =0
            y2 =image.shape[1]-1
            indices = (x1,y1,x2,y2)
            
          # Run detection
          else :
            results = model.detect([image], verbose=1)
            r = results[0]
            indi = r['rois'][r['class_ids']==15]
            if len(indi)==1:
              indices =indi[0]
            if (len(indi)==0):
              x1 =0
              x2 =image.shape[0]-1
              y1 =0
              y2 =image.shape[1]-1
              indices = (x1,y1,x2,y2)
              
            if (len(indi)>1) :
              ind = 0
              maxi = 0
              liste = r['scores'][r['class_ids']==15]
              for i in range(len(indi)):
                if liste[i] >=maxi:
                  maxi = liste[i]
                  ind = i
              indices = indi[ind]
            
            
          x1,y1,x2,y2 = indices
          img_crop = image[x1:x2,y1:y2]
          nfilename = '../bird_dataset_crop/'+x+'/'+label+'/'+im_name 
          if not os.path.exists(os.path.dirname(nfilename)):
            
            try:
              
              os.makedirs(os.path.dirname(nfilename))
            except OSError as exc: # Guard against race condition
              if exc.errno != errno.EEXIST:
                raise
          k+=1
          print(k)
          print(x)
          skimage.io.imsave(nfilename,img_crop)
            
            
                          
                          


Processing 1 images
image                    shape: (407, 500, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
1
test_images
Processing 1 images
image                    shape: (294, 500, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  134.20000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
2
test_images
Processing 1 images
image                    shape: (414, 500, 3)         min:    6.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    